# Handling missing values

In [1]:
# Load libraries
import pandas as pd
import numpy as np

In [12]:
# Load data
df_data = pd.read_csv('../data/raw/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
df_item = pd.read_csv('../data/raw/ml-100k/u.item', sep='|', header=None, names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], encoding='latin-1')
df_user = pd.read_csv('../data/raw/ml-100k/u.user', sep='|', header=None, names=['user_id','age','gender','occupation', 'zipcode'])

In [13]:
# Check for missing values in df_data
df_data.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [14]:
# Check for missing values in df_item
df_item.isnull().sum()

movie_id                 0
movie_title              0
release_date             1
video_release_date    1682
IMDb_URL                 3
unknown                  0
Action                   0
Adventure                0
Animation                0
Children                 0
Comedy                   0
Crime                    0
Documentary              0
Drama                    0
Fantasy                  0
Film-Noir                0
Horror                   0
Musical                  0
Mystery                  0
Romance                  0
Sci-Fi                   0
Thriller                 0
War                      0
Western                  0
dtype: int64

In [15]:
# Check for missing values in df_user
df_user.isnull().sum()

user_id       0
age           0
gender        0
occupation    0
zipcode       0
dtype: int64

In [16]:
# We see that there are no missing values in any of the dataframes except for df_item
# We know that there are 1682 movies in total, so we can just delete the column video_release_date
df_item.drop('video_release_date', axis=1, inplace=True)

In [17]:
# For the release_date, we can replace it with the median value, but beforehand we need to convert the release_date to a numeric value
df_item['release_date'] = pd.to_datetime(df_item['release_date'])
df_item['release_date'].fillna(df_item['release_date'].median(), inplace=True)

In [23]:
# Also, let's delete the IMDb_URL column as it seems to be irrelevant
df_item.drop('IMDb_URL', axis=1, inplace=True)

In [24]:
df_item.isnull().sum()

movie_id        0
movie_title     0
release_date    0
unknown         0
Action          0
Adventure       0
Animation       0
Children        0
Comedy          0
Crime           0
Documentary     0
Drama           0
Fantasy         0
Film-Noir       0
Horror          0
Musical         0
Mystery         0
Romance         0
Sci-Fi          0
Thriller        0
War             0
Western         0
dtype: int64

In [33]:
# Let's merge the dataframes
df = pd.merge(df_data, df_item, left_on='item_id', right_on='movie_id')
df = pd.merge(df, df_user, on='user_id')

In [34]:
# Let's encode the gender column
df['gender'] = df['gender'].apply(lambda x: 1 if x == 'M' else 0)
# df.head()
# Also, let's delete the movie_title
df.drop('movie_title', axis=1, inplace=True)
# And encode the occupation column as 0 or 1
df = pd.get_dummies(df, columns=['occupation'], dtype='int64')
df.head()

,user_id,item_id,rating,timestamp,movie_id,release_date,unknown,Action,Adventure,Animation,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,196,242,3,881250949,242,1997-01-24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,196,257,2,881251577,257,1997-07-04,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
2,196,111,4,881251793,111,1996-04-26,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,196,25,4,881251955,25,1996-03-08,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,196,382,4,881251843,382,1994-01-01,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [41]:
# Let's produce user-item matrix
user_item_matrix = df_data.drop('timestamp', axis=1)

In [43]:
user_item_matrix.to_csv('../data/interim/user_item_matrix.csv', index=False)